In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/consume-complaints-dataset-fo-nlp/complaints_processed.csv


In [2]:
import re
import nltk
from nltk.corpus import stopwords

In [3]:
df=pd.read_csv("/kaggle/input/consume-complaints-dataset-fo-nlp/complaints_processed.csv")

In [4]:
df.head()

,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


**Preprocess the text data**

In [5]:
def preprocess_text(text):
    # Convert to lowercase
    text=str(text)
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(text)
    text = [word for word in tokens if word not in stop_words]
    # Stem the words
    stemmer = nltk.PorterStemmer()
    text = [stemmer.stem(word) for word in text]
    # Convert back to string format
    text = ' '.join(text)
    return text

In [6]:
df['narrative'] = df['narrative'].apply(preprocess_text)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [8]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['narrative'])
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X)

In [9]:
y = df['product']

**Split the data into training and testing sets**

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

**Naive_Bayes**

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [13]:
# Train a Multinomial Naive Bayes model
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [14]:
# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


                     precision    recall  f1-score   support

        credit_card       0.79      0.58      0.67      3097
   credit_reporting       0.80      0.96      0.87     18271
    debt_collection       0.85      0.43      0.57      4627
mortgages_and_loans       0.81      0.78      0.79      3779
     retail_banking       0.90      0.73      0.80      2711

           accuracy                           0.81     32485
          macro avg       0.83      0.70      0.74     32485
       weighted avg       0.81      0.81      0.79     32485



**RandomForestClassifier**

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
RFC=RandomForestClassifier()
RFC.fit(X_train,y_train)

RandomForestClassifier()

In [17]:
y_pred_RFC=RFC.predict(X_test)

In [18]:
print(classification_report(y_test,y_pred_RFC))

                     precision    recall  f1-score   support

        credit_card       0.81      0.69      0.75      3097
   credit_reporting       0.88      0.98      0.93     18271
    debt_collection       0.90      0.69      0.78      4627
mortgages_and_loans       0.86      0.80      0.83      3779
     retail_banking       0.86      0.82      0.84      2711

           accuracy                           0.87     32485
          macro avg       0.86      0.80      0.83     32485
       weighted avg       0.87      0.87      0.87     32485

